In [107]:
import pandas as pd

In [108]:
df = pd.read_csv("../6m_mix.csv", nrows=1000)

In [109]:
import string
def split_punctuation(text):
    for i in string.punctuation:
        text = text.replace(i, " "+i+" ")
    text = " ".join(text.split())
    return text

In [110]:
df.head()

,text,summary,spell
0,Chây ì noojp phạt nguội .,Chây ì nộp phạt nguội .,"[0, 0, 1, 0, 0, 0]"
1,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,mây là á phương tiện vi phạm dc camera ( di dd...,Đây là các phương tiện vi phạm được camera ( d...,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
3,Phổ biến nhất là lỗi ỗ không đúng nơi quy định .,Phổ biến nhất là lỗi đỗ không đúng nơi quy định .,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14...","Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [111]:
df['text'] = df['text'].map(lambda x : split_punctuation(x))
df['summary'] = df['summary'].map(lambda x : split_punctuation(x))

In [112]:
lst_row = []
lst_summary = []
lst_spell = []
for idx, row in df.iterrows():
    if len(row['text'].split(' ')) != len(row['summary'].split(' ')):
        continue
    spell = []
    for i, j in zip(row['text'].split(' '), row['summary'].split(' ')):
        if i == j:
            spell.append(0)
        else:
            spell.append(1)
    lst_row.append(row['text'])
    lst_summary.append(row['summary'])
    lst_spell.append(spell)
    

df = pd.DataFrame({"text":lst_row, "spell":lst_spell, "summary": lst_summary})

In [113]:
df.head()

,text,spell,summary
0,Chây ì noojp phạt nguội .,"[0, 0, 1, 0, 0, 0]",Chây ì nộp phạt nguội .
1,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hàng chục ngàn phương tiện bị ghi hình vi phạm...
2,mây là á phương tiện vi phạm dc camera ( di dd...,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...",Đây là các phương tiện vi phạm được camera ( d...
3,Phổ biến nhất là lỗi ỗ không đúng nơi quy định .,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",Phổ biến nhất là lỗi đỗ không đúng nơi quy định .
4,"Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Chẳng hạn từ tháng 1 - 7 . 2017 , ô tô BS : 14..."


In [114]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from utils import *

class PhoBertDataset(Dataset):
    def __init__(self, indexs,
                 token_ids=None,
                 data=None,
                 is_train: bool = True,
                 ):
        self.indexs = indexs
        self.n_errors = 2
        self.mask_token_id = 0
        self.is_train = is_train
        if is_train:
            self.token_ids = token_ids
            self.correct_token_ids = self.token_ids['correct_token_ids']
            self.n_words = self.token_ids['n_words']
            self.error_ids = self.token_ids['error_ids']
            self.label_errors = self.token_ids['label_errors']
            self.size_subwords = self.token_ids['size_subwords']
        else:
            self.data = data
            self.word_error_tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
            self.error_ids = tokenizer_ids(self.data.error_sentence, self.word_error_tokenizer)
            self.size_subwords = calculate_size_subword(self.data.error_sentence, self.word_error_tokenizer)

    def __len__(self):
        return len(self.indexs)

    def __getitem__(self, idx):
        if self.is_train:
            idx = self.indexs[idx]
        data = {'error_id': torch.tensor(self.error_ids[idx], dtype=torch.long)}
        data['attention_mask'] = torch.tensor([0] + [1] * (data['error_id'].size(0) - 2) + [0], dtype=torch.long)
        data['batch_split'] = self.size_subwords[idx]
        if self.is_train:
            data['label_error'] = torch.tensor(self.label_errors[idx])
            data['word_correction'] = torch.tensor(self.correct_token_ids[idx])
        return data


In [115]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

In [116]:
from datasets import Dataset as dts

In [117]:
dataset = dts.from_pandas(df)


In [118]:
dataset

Dataset({
    features: ['text', 'spell', 'summary'],
    num_rows: 969
})

In [120]:
def mapping_tokenizer(examples):
    examples['correct_token_ids'] = tokenizer(examples['summary'])['input_ids']
    examples['error_ids'] = tokenizer(examples['text'])['input_ids']
    examples['label_errors'] = [0] + eval(str(examples['spell'])) + [0]
    del examples['spell']
    examples['batch_split'] = [1] + calculate_size_subword([examples['text']], tokenizer)[0] + [1]
    return examples

dataset = dataset.map(mapping_tokenizer, batched=False, num_proc=8)

Map (num_proc=8):   0%|          | 0/969 [00:00<?, ? examples/s]

In [121]:
len(dataset[1]['error_ids'])

33

In [122]:
calculate_size_subword(['<s> Trong 5 thập kỷ qua, dân số thị trấn này đã giảm gần một nửa, tư2 2, 25 0 ngườo xuống chỉ còn 1. 30 0 người do số lượng trẻ sơ sinh quá thấp. </s>'], tokenizer)[0]

[1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1]

In [83]:
tokenizer.tokenize('<s> Trong 5 thập kỷ qua, dân số thị trấn này đã giảm gần một nửa, tư2 2, 25 0 ngườo xuống chỉ còn 1. 30 0 người do số lượng trẻ sơ sinh quá thấp. </s>')

['<s>',
 'Trong',
 '5',
 'thập',
 'kỷ',
 'qua@@',
 ',',
 'dân',
 'số',
 'thị',
 'trấn',
 'này',
 'đã',
 'giảm',
 'gần',
 'một',
 'nử@@',
 'a@@',
 ',',
 'tư@@',
 '2',
 '2@@',
 ',',
 '25',
 '0',
 'ngườ@@',
 'o',
 'xuống',
 'chỉ',
 'còn',
 '1.',
 '30',
 '0',
 'người',
 'do',
 'số',
 'lượng',
 'trẻ',
 'sơ',
 'sinh',
 'quá',
 'thấ@@',
 'p.',
 '</s>']

In [123]:
format_ds = {"error_ids":dataset['error_ids'],"label_errors":dataset['label_errors'],"size_subwords":dataset['batch_split']}

In [125]:
import pickle

with open('sample_dataset.pkl', 'wb') as handle:
    pickle.dump(format_ds, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
sum([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

16

In [127]:
len(format_ds['error_ids'])

969

In [134]:
for idx, i in enumerate(format_ds['error_ids']):
    if i == [   0,  5113, 32000,    13,   963, 28350,  5855,    57,  4616,   611,
           18,    85,  4729,   327, 5, 2]:
        print(idx)

637


In [138]:
len(dataset[637]['error_ids'])

16

In [132]:
sum([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

33

In [139]:
import torch
pos_weight = torch.randint(1, 100, (1,)).float()

In [140]:
pos_weight

tensor([80.])